# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [16]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [5]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[int(num_test/2):], newsgroups_test.target[int(num_test/2):]
dev_data, dev_labels = newsgroups_test.data[:int(num_test/2)], newsgroups_test.target[:int(num_test/2)]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [4]:
def P1(num_examples=5):
### STUDENT START ###
    # iterate through the specified first number of examples and print them out
    for i in range(num_examples):
        print('Example %d, Label:' %(i+1), newsgroups_train.target_names[train_labels[i]])
        print('Text:\n', train_data[i])

        print('-'*50, '\n')

### STUDENT END ###
P1()

Example 1, Label: comp.graphics
Text:
 Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
-------------------------------------------------- 

Example 2, Label: talk.religion.misc
Text:
 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcak

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [5]:
def P2():
### STUDENT START ###


    v1 = CountVectorizer() # create an object of standard CountVectorizer
    data_matrix = v1.fit_transform(train_data) # get the sparse matrix of the transformed train data
    num_examples, num_features = data_matrix.shape # get the number of example and features

    ############### answers to question a.

    print('Answers to question a:')

    # get size of vocabulary
    print('\tThe size of the vocabulary is:', num_features)

    # get the average nonzero feature count per example
    print('\tThe average nonzero feature count per example: %5.2f' %(data_matrix.nnz/num_examples))

    # get the fraction of the entries in the matrix are non-zero
    print('\tThe fraction of the entries in the matrix are non-zero: %1.5f' %(data_matrix.nnz/(num_examples * num_features)))

    print('\n')

    ############### answers to question b.

    feature_names = v1.get_feature_names()

    print('Answers to question b:')
    # print the 0th and last feature strings
    print('\tThe 0th feature:', feature_names[0])
    print('\tThe last feature:', feature_names[-1])
    print('\n')


    ############### answers to question c.

    # build my vocab with the four words
    my_vocab = ["atheism", "graphics", "space", "religion"]
    # fit the vocab
    v1.fit(my_vocab)
    # transform the data with the model of the vocab fitted
    my_vocab_data = v1.transform(train_data)
    print('Answers to question c:')
    # confirm the shape
    print('\tThe shape of the transformed data is:', my_vocab_data.shape)
    print('\tThe shape is expected since we have four words in our vocab and the same number of examples used in training')
    print('\tThe average nonzero feature count per example: %5.2f' %(my_vocab_data.nnz/num_examples))
    print('\n')

    ############### answers to question d.

    # generate a vectorizer for bi-gram and tri-gram character features
    v2 = CountVectorizer(analyzer = 'char', ngram_range = (2,3))
    # fit the training data
    v2.fit(train_data)
    # print answer
    print('Answers to question d:')
    print('\tVocab size is:', len(v2.get_feature_names()))
    print('\n')

    ############### answers to question e.
    # generate a vectorizer for with the cut-off of 10
    v3 = CountVectorizer(min_df = 10)
    v3.fit(train_data)
    # print answer
    print('Answers to question e:')
    print('\tVocab size of only the words appearing in at least ten documents is:', len(v3.get_feature_names()))
    print('\n')

    ############### answers to question f.
    # fit dev data and get the features
    v1.fit(dev_data)
    dev_features = v1.get_feature_names()

    # fit train data again
    v1.fit(train_data)

    # identify the features in dev but not in train
    not_in_train = 0
    for feature in dev_features:
        if v1.vocabulary_.get(feature) == None:
            not_in_train += 1
    # print answers
    print('Answers to question f:')
    print('\tThere are %d words in the dev data, out of which, %d do not exist in the train data.' %(len(dev_features), not_in_train))
    print('\tThat is %3.1f%%' %(not_in_train/len(dev_features)*100))

### STUDENT END ###
P2()

Answers to question a:
	The size of the vocabulary is: 26879
	The average nonzero feature count per example: 96.71
	The fraction of the entries in the matrix are non-zero: 0.00360


Answers to question b:
	The 0th feature: 00
	The last feature: zyxel


Answers to question c:
	The shape of the transformed data is: (2034, 4)
	The shape is expected since we have four words in our vocab and the same number of examples used in training
	The average nonzero feature count per example:  0.27


Answers to question d:
	Vocab size is: 35478


Answers to question e:
	Vocab size of only the words appearing in at least ten documents is: 3064


Answers to question f:
	There are 16246 words in the dev data, out of which, 4027 do not exist in the train data.
	That is 24.8%


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [29]:
def P3():
### STUDENT START ###

    # initialize the vectorizer and transform the train and dev data
    default_vectorizer = CountVectorizer()
    train_matrix = default_vectorizer.fit_transform(train_data)
    dev_matrix = default_vectorizer.transform(dev_data)
    
    # provide a range of k to optimize on
    ks = [1,2,3,4,5,6,7,8,9,10,15,20,30,35,40,45,50,60,70,80, 100, 200, 300]
    
    # fit the KNN models with different k values and make predictions 
    f1_scores = []
    for k in ks:
        clf_knn = KNeighborsClassifier(n_neighbors = k)
        clf_knn.fit(train_matrix, train_labels)
        predictions = clf_knn.predict(dev_matrix)
        # collect f1 scores
        f1_scores.append(metrics.f1_score(dev_labels, predictions, average = 'macro'))
    # find the optimal f1 scores
    max_f1 = 0
    optimal_k = 0
    for i in range(len(ks)):
        if f1_scores[i] > max_f1:
            max_f1 = f1_scores[i]
            optimal_k = ks[i]
            
    # output the results
    print('KNN models: ', '\n')
    print('After searching through k =', ks)
    print('The optimal k is:', optimal_k, 'and the f1 score is:', round(max_f1,2))
    print('-'*50)
    
    
    # provide a range of alphas to optimize on
    alphas = [0.000001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
    
    # fit the naive bayes models with different alpha values and make predictions 
    f1_scores = []
    for alpha in alphas:
        mnb = MultinomialNB(alpha = alpha)
        mnb.fit(train_matrix, train_labels)
        predictions = mnb.predict(dev_matrix)

        f1_scores.append(metrics.f1_score(dev_labels, predictions, average = 'macro'))
    
    # find the optimal f1 scores
    max_f1 = 0
    optimal_alpha = 0
    for i in range(len(alphas)):
        if f1_scores[i] > max_f1:
            max_f1 = f1_scores[i]
            optimal_alpha = alphas[i]    
            
    # output the results
    print('Naive Bayes models: ', '\n')
    print('After searching through alpha =', alphas)
    print('The optimal alpha is:', optimal_alpha, 'and the f1 score is:', round(max_f1,2))
    print('-'*50)
    
    # provide a range of c to optimize on
    cs = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 50.0, 100.0, 500.0, 1000.0]

    # fit the logistic regression models with different c values and make predictions, also collect the sum of sqaures for different cd 
    f1_scores = []
    sums_of_squares = [] 
    for c in cs:
        logist_regress = LogisticRegression(C = c)
        logist_regress.fit(train_matrix, train_labels)
        predictions = logist_regress.predict(dev_matrix)

        f1_scores.append(metrics.f1_score(dev_labels, predictions, average = 'macro'))
        sums_of_squares.append(np.sum(np.square(logist_regress.coef_), axis = 1))
    
    # search for the optimal c
    max_f1 = 0
    optimal_c = 0
    for i in range(len(cs)):
        if f1_scores[i] > max_f1:
            max_f1 = f1_scores[i]
            optimal_c = cs[i] 
            
    # output the results
    print('After searching through c =', cs)
    print('The optimal c is:', optimal_c, 'and the f1 score is:', round(max_f1,5))
    print('-'*50)
    
    for c, sum_of_squares in zip(cs, sums_of_squares):
        print('When C = %-8s,' %str(c), 'sum of squared parameters =', sum_of_squares)
    
### STUDENT END ###
P3()

KNN models:  

After searching through k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 35, 40, 45, 50, 60, 70, 80, 100, 200, 300]
The optimal k is: 40 and the f1 score is: 0.44
--------------------------------------------------
Naive Bayes models:  

After searching through alpha = [1e-06, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
The optimal alpha is: 0.1 and the f1 score is: 0.77
--------------------------------------------------


C:\Users\erik.hou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\erik.hou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


After searching through c = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 50.0, 100.0, 500.0, 1000.0]
The optimal c is: 0.5 and the f1 score is: 0.69202
--------------------------------------------------
When C = 0.0001  , sum of squared parameters = [0.00770175 0.0119412  0.00943508 0.00910284]
When C = 0.001   , sum of squared parameters = [0.16509345 0.20095275 0.18067094 0.18724278]
When C = 0.01    , sum of squared parameters = [2.54149597 2.93970937 2.86246884 2.25002867]
When C = 0.1     , sum of squared parameters = [27.13276422 24.65876272 27.45791178 23.02092251]
When C = 0.5     , sum of squared parameters = [102.60222594  83.1201744   99.01364429  88.98434638]
When C = 1.0     , sum of squared parameters = [166.96406144 130.8904599  157.88282512 145.71125673]
When C = 2.0     , sum of squared parameters = [257.63081206 197.94301097 239.9678541  226.71318397]
When C = 10.0    , sum of squared parameters = [584.53080597 448.08852555 539.41970751 530.78234875]
When C = 50.0 

ANSWER:

1. Because the feature space is too large(the size of the vocabulary is 26879), as a result, KNN algorithm performs poorly. Due to the fact that in such a giant feature space, all observations are very far from each other. No obvious clusters associated with labels. Plus, by just doing the raw count, the common uninformative words like 'is', 'are', 'were' etc... will wash out the effect of important words associated with labels.


2. Because logistic regression is a linear classifier, with a sparse feature space like what we have, it makes it more difficult to classify the examples.


3. We can see that as C increases, the sum of squred parameters increases. It's because C is the inverse of the regularization coefficient. The lower C is, the heavier the regularization which leads to heavier panelty for larger parameters.


(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [65]:
def print_table(data, column_labels, row_labels, first_col_width):
    print(' '*first_col_width, end=' ')
    for col_label in column_labels:
        print('%-10s' %col_label, end=' ')
    print('\n')
    
    for row_label, data in zip(row_labels, data):
        print('%-0*s' %(first_col_width, row_label), end=' ')
        for datum in data:
            print('%-10.3f' %datum, end=' ')
        print('\n')

In [8]:
#def P4():
### STUDENT START ###
c_vectorizer = CountVectorizer() 
train_matrix = c_vectorizer.fit_transform(train_data)

logist_regress = LogisticRegression(C = 0.1)
logist_regress.fit(train_matrix, train_labels)
params = logist_regress.coef_

index = np.argsort(params, axis = 1)[:,-5:].reshape(20,)

trans_params = np.transpose(params[:,index])
feature_names = c_vectorizer.get_feature_names()
labels = np.unique(train_labels)
features = [feature_names[i] for i in index]

print_table(trans_params, labels, features, 15)


print('-'*50)
print('\n\n')

c_vectorizer = CountVectorizer(ngram_range = (2,2)) 
train_matrix = c_vectorizer.fit_transform(train_data)

logist_regress = LogisticRegression(C = 0.1)
logist_regress.fit(train_matrix, train_labels)
params = logist_regress.coef_

index = np.argsort(params, axis = 1)[:,-5:].reshape(20,)

trans_params = np.transpose(params[:,index])
feature_names = c_vectorizer.get_feature_names()
labels = np.unique(train_labels)
features = [feature_names[i] for i in index]

print_table(trans_params, labels, features, 15)

# dev_matrix = c_vectorizer.transform(dev_data)
# predictions = logist_regress.predict(dev_matrix)
# print(metrics.f1_score(dev_labels, predictions, average = 'macro'))

### STUDENT END ###
#P4()

                0          1          2          3          

islam           0.426      -0.085     -0.165     -0.165     

atheists        0.461      -0.079     -0.158     -0.295     

bobby           0.478      -0.120     -0.168     -0.228     

religion        0.494      -0.299     -0.393     0.004      

atheism         0.495      -0.207     -0.200     -0.268     

3d              -0.182     0.547      -0.312     -0.181     

computer        -0.040     0.559      -0.329     -0.229     

file            -0.177     0.641      -0.422     -0.288     

image           -0.264     0.642      -0.368     -0.216     

graphics        -0.411     1.008      -0.651     -0.372     

moon            -0.203     -0.291     0.403      -0.056     

launch          -0.211     -0.245     0.479      -0.169     

nasa            -0.273     -0.262     0.541      -0.253     

orbit           -0.218     -0.334     0.597      -0.249     

space           -0.655     -0.714     1.259      -0.590     

fbi     

ANSWER:

When using only unigram features, we can see the clear association between label and the features with large weight. However, when using only bigram features, many of the features included don't seem to have apparent link with the labels semantically. They are included probably because they just happen to appear more often in the training set when conditioned on label.

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

countvectorizer automatically lowercase everything.

In [38]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
### STUDENT START ###
    
    # set a pattern that takes out all the non alphanumeric characters
    non_alphanumeric = re.compile(r'[^A-Za-z0-9]')
    s = non_alphanumeric.sub(' ', s)
    
    # not replace digits with a single token because it decrease performance
#     digit_sequence = re.compile(r'\d{1,3}')
    
    # lower case everything
    s = s.lower()
    
    # when words are longer than 8 character, only get the first eight chars
    long_words = re.compile(r'\w{8,}')
    
    longs = long_words.findall(s)
    s = long_words.sub('', s)
    for long in longs:
        s = s + ' ' + long[:8]

    return s

### STUDENT END ###

def P5():
### STUDENT START ###
    # calculate the original vocab size
    c_vectorizer = CountVectorizer()
    train_matrix = c_vectorizer.fit_transform(train_data)
    original_size = train_matrix.shape[1]
    
    # vectorize the train_ata with preprocessor
    c_vectorizer = CountVectorizer(preprocessor = better_preprocessor) 
    train_matrix = c_vectorizer.fit_transform(train_data)
    with_prepro_size = train_matrix.shape[1]
    
    # output the new vocab size and comparison
    print('The new vocab size is:', with_prepro_size)
    print('It reduce the size of vocab compared with the model with no preprocessor by:', original_size-with_prepro_size)
    print('-'*50, '\n')
    
    
    #transform dev data
    dev_matrix = c_vectorizer.transform(dev_data)
    
    cs = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 50.0, 100.0, 500.0, 1000.0]

    f1_scores = []
    sums_of_squares = [] 
    for c in cs:
        logist_regress = LogisticRegression(C = c, max_iter = 500)
        logist_regress.fit(train_matrix, train_labels)
        predictions = logist_regress.predict(dev_matrix)

        f1_scores.append(metrics.f1_score(dev_labels, predictions, average = 'macro'))
        sums_of_squares.append(np.sum(np.square(logist_regress.coef_), axis = 1))

    max_f1 = 0
    optimal_c = 0
    for i in range(len(cs)):
        if f1_scores[i] > max_f1:
            max_f1 = f1_scores[i]
            optimal_c = cs[i]    

    print('After searching through c =', cs)
    print('The optimal c is:', optimal_c, 'and the f1 score is:', round(max_f1,5))
    print('-'*50)
    
### STUDENT END ###
P5()

The new vocab size is: 23853
It reduce the size of vocab compared with the model with no preprocessor by: 3026
-------------------------------------------------- 

After searching through c = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 50.0, 100.0, 500.0, 1000.0]
The optimal c is: 0.5 and the f1 score is: 0.70247
--------------------------------------------------


Compared with the original performance, with preprossed features, the performance improved.

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [151]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    c_vectorizer = CountVectorizer() 
    
    
    cs = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 500.0, 1000.0, 5000.0, 10000.0, 50000.0, 100000, 500000]

#     accuracies = []
#     for c in cs:
    for c in cs:
        print(c)
        
        train_matrix = c_vectorizer.fit_transform(train_data)
        
        logist_regress_l1 = LogisticRegression(C = c, penalty = 'l1', tol=.01)
        logist_regress_l1.fit(train_matrix, train_labels)

        logist_regress_l2 = LogisticRegression(C = c, tol=.01)
        logist_regress_l2.fit(train_matrix, train_labels)

        params_l1 = logist_regress_l1.coef_
        index_l1 = np.where(np.sum(params_l1, axis = 0) != 0)

        params_l2 = logist_regress_l2.coef_
        index_l2 = np.where(np.sum(params_l2, axis = 0) != 0)

        print(index_l1[0].size)
        print(index_l2[0].size)


        feature_names = c_vectorizer.get_feature_names()

        vocab = []
        for index in index_l1[0]:
            vocab.append(feature_names[index])

    #     print(vocab)

        c_vectorizer.fit(vocab)

        train_matrix = c_vectorizer.transform(train_data)

        logist_regress_l2 = LogisticRegression(C = c, tol=.01)
        logist_regress_l2.fit(train_matrix, train_labels)

        dev_matrix =  c_vectorizer.transform(dev_data)
        predictions = logist_regress_l2.predict(dev_matrix)
        print(metrics.f1_score(dev_labels, predictions, average = 'macro'))
        print(metrics.accuracy_score(dev_labels, predictions))
        print('-'*50)

    ### STUDENT END ###
P6()

0.001
2
26879
0.2866650989981874
0.34171597633136097
--------------------------------------------------
0.005
13
26879
0.4288709022052392
0.46745562130177515
--------------------------------------------------
0.01
18
26879
0.44327454812409034
0.48520710059171596
--------------------------------------------------
0.05
122
26879
0.5959500149585182
0.628698224852071
--------------------------------------------------
0.1
208
26879
0.6559146288312321
0.6804733727810651
--------------------------------------------------
0.5
732
26879
0.6612738362120769
0.6834319526627219
--------------------------------------------------
1.0


C:\Users\erik.hou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


1072
26879
0.6659615604531361
0.6878698224852071
--------------------------------------------------
5.0
2221
26879
0.6757180975629119
0.6937869822485208
--------------------------------------------------
10.0
3268
26879
0.6718537586647823
0.6908284023668639
--------------------------------------------------
50.0
5798
26879
0.668758081379605
0.6878698224852071
--------------------------------------------------
100.0
7753
26879
0.6692914758029448
0.6863905325443787
--------------------------------------------------
500.0
19120
26879
0.6753726158180405
0.6937869822485208
--------------------------------------------------
1000.0
23631
26879
0.6866425740202451
0.7085798816568047
--------------------------------------------------
5000.0
26729
26879
0.6814578311156905
0.7026627218934911
--------------------------------------------------
10000.0
26826
26879
0.6713985788113694
0.6923076923076923
--------------------------------------------------
50000.0
26871
26879
0.6767709720900925
0.69378698

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [275]:
#def P7():
    ### STUDENT START ###
    
tfidf_v = TfidfVectorizer()
train_matrix = tfidf_v.fit_transform(train_data)

dev_matrix = tfidf_v.transform(dev_data)

logist_regress = LogisticRegression(C = 100)
logist_regress.fit(train_matrix, train_labels)
predictions = logist_regress.predict(dev_matrix)

probs = logist_regress.predict_proba(dev_matrix)
params = logist_regress.coef_.T


rs = []
for i in range(len(dev_labels)):
    prob = probs[i]
    dev_label = dev_labels[i]
    rs.append(np.amax(prob)/prob[dev_label])

indicies = np.argsort(rs)[-3:]

for index in indicies:
    print('Ratio, R:', round(rs[index],2))
    print('Predicted Label: %s,%s' %(predictions[index], newsgroups_train.target_names[predictions[index]]))
    print('True Label: %s,%s' %(dev_labels[index],newsgroups_train.target_names[dev_labels[index]]))
    print('\n')
    
#     print('Data Content:')
    print(dev_data[index])
    print('\n')
    array = dev_matrix[index,:].toarray()
    tf_idf_thetas = []
    tf_idf_theta_max = []
    sort_col = []
    rows = []
    for i in range(array.shape[1]):
        if array[0][i] != 0:
            
            sort_index = (array[0][i]*params[i])[predictions[index]]-(array[0][i]*params[i])[dev_labels[index]]
            sort_col.append(sort_index)

            
            rows.append((
#                          i, 
                         tfidf_v.get_feature_names()[i], 
                         np.round(tfidf_v.idf_[i],3),
                         np.round(array[0][i],3), 
                         np.round(array[0][i]*params[i],2), 
                         sort_index))
            
            tf_idf_thetas.append(np.round(array[0][i]*params[i],2))

    sorted_indices = np.argsort(sort_col)
    
    print('%12s%10s%10s%30s%15s' %('feature', 'idf', 'tf-idf', 'tf-idf*thetas', 'input diff'))
    for pos in reversed(sorted_indices):
        if np.max(np.absolute(rows[pos][3])) >= 0.05:
            print('%12s%10.3f%10.3f%30s%15.2f' %rows[pos])
    
    print('%32s%30s' %('SUM', np.sum(tf_idf_thetas, axis=0)))
    print('-'*50, '\n')
    
    
con_matrix = confusion_matrix(dev_labels, predictions, labels = np.sort(np.unique(dev_labels)))
#     print(confusion)
print(con_matrix)
    ### STUDENT END ###
#P7()

Ratio, R: 287.31
Predicted Label: 3,talk.religion.misc
True Label: 0,alt.atheism



The 24 children were, of course, killed by a lone gunman in a second story
window, who fired eight bullets in the space of two seconds...



     feature       idf    tf-idf                 tf-idf*thetas     input diff
    children     4.611     0.235     [-1.01 -0.4  -0.39  1.33]           2.34
       story     4.857     0.248     [-0.33 -0.55 -0.26  1.19]           1.52
      second     4.130     0.211     [-0.65  0.08 -0.17  0.6 ]           1.25
         two     3.320     0.169     [-0.71  0.29 -0.26  0.49]           1.20
          by     2.200     0.112     [-0.21 -0.09 -0.14  0.17]           0.38
        were     2.819     0.144     [ 0.06 -0.36 -0.32  0.42]           0.36
         who     2.624     0.134     [ 0.2  -0.65 -0.31  0.52]           0.32
         the     1.189     0.121     [-0.35 -0.46  0.28 -0.06]           0.29
      killed     5.122     0.261     [ 0.1  -0.29 -0.26  0.35]           

ANSWER:

The TfidfVectorizer(term frequency times inverse document frequency) is a vectorizer that takes into consideration not only the frequency of a term appearing in the training corpus, but how often it appears in different documents in the corpus. When vectorizing the training data, it penalizes the terms having high frequency everywhere across the training data since those words shouldn't be given mush weights in training the classifier because they are not strongly associated with a particular label. 

The score of a term for a particular example is:

Term Frequency in the example times IDF, where IDF = log[n/df(t)] + 1, n = total number of examples, df(t)= the number of documents in which the term appears.

I built the model that is fitted to the vocabulary vectorized by TfidfVectorizer and predicted the dev data. Examined the three examples having highest R values. For each example, a table that has the features and its relevant information is generated. The table is sorted by the difference between feature's input into the one vs all regression model used by the predicted label and the true label; therefore, the bigger the difference, the more that feature is driving the model towards the prediction and away from the true label.

I noticed a few things in the three examples that had high R values and some suggestions for further steps to address the issues:

* These are typically the documents that includes some key words for the predicted labels but not having enough words associated with the true label. If those words happen to have high idf, such as ftp (example 2 and 3) and children(example 1), they can drive the prediction in the wrong direction quickly.

* Cross reference the confusion matrix generated, we see that the model does a decent job predicting label 1 and 2, but struggles to predict label 3 correctly. It's also the case that we have the least training data on label three. Though TF-IDF already gives more weights to words that appear less which helps the minority class in our training data, I think adding more training data might help with the overall accuracy.

* I think it might also help if we take out some words that drives the model away from label 3, such as ftp since the model performs very well in predicting label 1 already.

* Some stop words that doesn't contain label specific information such as, "by", "were", and "any" were found to have big impact on the predictions in both directions (it drives the predictions to and from the true label when they don't seem to have clear semantic relationship with the labels). By taking them out, it may improve performance.


(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance

Refering back to Part 7., will exclude the stop words and see if the performance of the model improved.

In [22]:
tfidf_v = TfidfVectorizer()
train_matrix = tfidf_v.fit_transform(train_data)

dev_matrix = tfidf_v.transform(dev_data)

logist_regress = LogisticRegression(C = 100)
logist_regress.fit(train_matrix, train_labels)
predictions = logist_regress.predict(dev_matrix)

report = classification_report(dev_labels, predictions)
print('Original Model F1-Score:', round(metrics.f1_score(dev_labels, predictions, average = 'macro'),3))
print('Original Model Accuracy:', round(metrics.accuracy_score(dev_labels, predictions), 3))
print('-'*50, '\n')
tfidf_v = TfidfVectorizer(stop_words = 'english')
train_matrix = tfidf_v.fit_transform(train_data)

dev_matrix = tfidf_v.transform(dev_data)

logist_regress = LogisticRegression(C = 100)
logist_regress.fit(train_matrix, train_labels)
predictions = logist_regress.predict(dev_matrix)

print('Stop Words Excluded Model F1-Score:', round(metrics.f1_score(dev_labels, predictions, average = 'macro'),3))
print('Stop Words Excluded Original Model Accuracy:', round(metrics.accuracy_score(dev_labels, predictions), 3))

Original Model F1-Score: 0.747
Original Model Accuracy: 0.763
-------------------------------------------------- 

Stop Words Excluded Model F1-Score: 0.763
Stop Words Excluded Original Model Accuracy: 0.781


By excluding stop words, there is a improvement on both f1 score and accuracy.